In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_627074/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        self.discard_below_average(sort_by="r2")
        self.discard_below_average(sort_by="r2_vt")
        self.discard_high_standard_deviation()
        self.results_df.to_excel(f"better_results.xlsx", index=True)
        display(self.results_df)


In [6]:
analize = Analizer(0.9)
analize.Analize()
analize.clean_folder(subfolder="dataset", extension="pkl")
analize.clean_folder(subfolder="results", extension="xlsx")
analize.clean_folder(subfolder="results", extension="txt")
analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,r2_vt,mse,mse_sup,mse_test,mse_val,mse_vt,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_38_9_24,0.999962,0.770471,0.999977,0.999626,0.999909,0.000054,0.325783,0.000063,0.000252,0.000158,0.006553,0.007377,1.000001,0.007691,1689.637512,2707.398826,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
1,model_38_9_23,0.999962,0.770487,0.999977,0.999632,0.999911,0.000055,0.325760,0.000063,0.000248,0.000156,0.006592,0.007392,1.000001,0.007707,1689.629356,2707.390670,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
2,model_38_9_22,0.999961,0.770505,0.999977,0.999639,0.999912,0.000055,0.325735,0.000063,0.000243,0.000153,0.006632,0.007418,1.000001,0.007734,1689.615485,2707.376799,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
3,model_38_9_21,0.999961,0.770524,0.999977,0.999647,0.999913,0.000056,0.325707,0.000064,0.000238,0.000151,0.006682,0.007457,1.000001,0.007775,1689.594315,2707.355629,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
4,model_38_9_20,0.999960,0.770546,0.999977,0.999656,0.999915,0.000056,0.325677,0.000064,0.000232,0.000148,0.006735,0.007514,1.000001,0.007834,1689.563863,2707.325176,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099,model_34_9_20,0.995228,0.763050,0.993376,0.992750,0.992883,0.006773,0.336316,0.004874,0.037455,0.021165,0.030886,0.082297,1.000141,0.085800,1679.989697,2697.751010,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
1100,model_34_9_24,0.995228,0.763050,0.993376,0.992750,0.992883,0.006773,0.336316,0.004874,0.037455,0.021165,0.030886,0.082297,1.000141,0.085800,1679.989696,2697.751010,"Hidden Size=[33, 21], regularizer=0.2, learnin..."
1101,model_14_7_3,0.995194,0.703645,0.988733,0.991829,0.992615,0.006822,0.420633,0.008328,0.006501,0.007415,0.190885,0.082594,1.000923,0.086111,307.975252,489.587750,"Hidden Size=[20, 4], regularizer=0.2, learning..."
1107,model_20_7_2,0.995102,0.811133,0.991353,0.992590,0.995574,0.006951,0.268069,0.001910,0.003106,0.002508,0.171602,0.083375,1.000452,0.086924,577.937628,924.098362,"Hidden Size=[15, 14], regularizer=0.0, learnin..."
